# Importar Librarias y datos

In [152]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np   
import itertools

from scipy.stats import chi2_contingency, ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf 

In [153]:
df = pd.read_parquet("output_01.parquet")

In [208]:
df.head()

,ID,age,gender,education,class,education_institute,marital_status,race,is_hispanic,employment_commitment,...,country_of_birth_mother,migration_code_change_in_msa,migration_prev_sunbelt,migration_code_move_within_reg,migration_code_change_in_reg,residence_1_year_ago,old_residence_reg,old_residence_state,importance_of_record,income_above_limit
0,ID_TZ0000,79,Female,High school graduate,None,None,Widowed,White,All other,Not in labor force,...,US,?,?,?,?,None,None,None,1779.74,Below limit
1,ID_TZ0001,65,Female,High school graduate,None,None,Widowed,White,All other,Children or Armed Forces,...,US,unchanged,None,unchanged,unchanged,Same,None,None,2366.75,Below limit
2,ID_TZ0002,21,Male,12th grade no diploma,Federal government,None,Never married,Black,All other,Children or Armed Forces,...,US,unchanged,None,unchanged,unchanged,Same,None,None,1693.42,Below limit
3,ID_TZ0003,2,Female,Children,None,None,Never married,Asian or Pacific Islander,All other,Children or Armed Forces,...,India,unchanged,None,unchanged,unchanged,Same,None,None,1380.27,Below limit
4,ID_TZ0004,70,Male,High school graduate,None,None,Married-civilian spouse present,White,All other,Not in labor force,...,US,?,?,?,?,None,None,None,1580.79,Below limit


# Corregir nombres de columnas

In [155]:
# Cambiar el nombre de una columna en especifico
df.rename(columns={'mig_year': 'migration_year'}, inplace=True)

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209499 entries, 0 to 209498
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              209499 non-null  object 
 1   age                             209499 non-null  int64  
 2   gender                          209499 non-null  object 
 3   education                       209499 non-null  object 
 4   class                           104254 non-null  object 
 5   education_institute             13302 non-null   object 
 6   marital_status                  209499 non-null  object 
 7   race                            209499 non-null  object 
 8   is_hispanic                     209499 non-null  object 
 9   employment_commitment           209499 non-null  object 
 10  unemployment_reason             6520 non-null    object 
 11  employment_stat                 209499 non-null  int64  
 12  wage_per_hour   

# Corregir valores categóricos

In [158]:
# Identificar columnas categóricas
columnas_categoricas = df.select_dtypes(include=['object']).columns

In [159]:
columnas_categoricas

Index(['ID', 'gender', 'education', 'class', 'education_institute',
       'marital_status', 'race', 'is_hispanic', 'employment_commitment',
       'unemployment_reason', 'is_labor_union', 'industry_code_main',
       'occupation_code_main', 'household_stat', 'household_summary',
       'under_18_family', 'veterans_admin_questionnaire', 'tax_status',
       'citizenship', 'country_of_birth_own', 'country_of_birth_father',
       'country_of_birth_mother', 'migration_code_change_in_msa',
       'migration_prev_sunbelt', 'migration_code_move_within_reg',
       'migration_code_change_in_reg', 'residence_1_year_ago',
       'old_residence_reg', 'old_residence_state', 'income_above_limit'],
      dtype='object')

In [160]:
# Al revisar los valores vemos que tienen espacios en blanco
for col in columnas_categoricas:
    print(f"Valores únicos en '{col}': {df[col].unique()}")

Valores únicos en 'ID': ['ID_TZ0000' 'ID_TZ0001' 'ID_TZ0002' ... 'ID_TZ99997' 'ID_TZ99998'
 'ID_TZ99999']
Valores únicos en 'gender': [' Female' ' Male']
Valores únicos en 'education': [' High school graduate' ' 12th grade no diploma' ' Children'
 ' Bachelors degree(BA AB BS)' ' 7th and 8th grade' ' 11th grade'
 ' 9th grade' ' Masters degree(MA MS MEng MEd MSW MBA)' ' 10th grade'
 ' Associates degree-academic program' ' 1st 2nd 3rd or 4th grade'
 ' Some college but no degree' ' Less than 1st grade'
 ' Associates degree-occup /vocational'
 ' Prof school degree (MD DDS DVM LLB JD)' ' 5th or 6th grade'
 ' Doctorate degree(PhD EdD)']
Valores únicos en 'class': [None ' Federal government' ' Private' ' Local government'
 ' Self-employed-incorporated' ' Self-employed-not incorporated'
 ' State government' ' Without pay' ' Never worked']
Valores únicos en 'education_institute': [None ' High school' ' College or university']
Valores únicos en 'marital_status': [' Widowed' ' Never married' ' Mar

In [161]:
# Aplicar correcciones a todas las columnas categóricas quitando todos los espacios en blanco
df[columnas_categoricas] = df[columnas_categoricas].apply(
    lambda col: col.str.strip()
)

In [162]:
# Volver a revisar valores únicos de todas las columnas categóricas
for col in columnas_categoricas:
    print(f"Valores únicos en '{col}': {df[col].unique()}")

Valores únicos en 'ID': ['ID_TZ0000' 'ID_TZ0001' 'ID_TZ0002' ... 'ID_TZ99997' 'ID_TZ99998'
 'ID_TZ99999']
Valores únicos en 'gender': ['Female' 'Male']
Valores únicos en 'education': ['High school graduate' '12th grade no diploma' 'Children'
 'Bachelors degree(BA AB BS)' '7th and 8th grade' '11th grade' '9th grade'
 'Masters degree(MA MS MEng MEd MSW MBA)' '10th grade'
 'Associates degree-academic program' '1st 2nd 3rd or 4th grade'
 'Some college but no degree' 'Less than 1st grade'
 'Associates degree-occup /vocational'
 'Prof school degree (MD DDS DVM LLB JD)' '5th or 6th grade'
 'Doctorate degree(PhD EdD)']
Valores únicos en 'class': [None 'Federal government' 'Private' 'Local government'
 'Self-employed-incorporated' 'Self-employed-not incorporated'
 'State government' 'Without pay' 'Never worked']
Valores únicos en 'education_institute': [None 'High school' 'College or university']
Valores únicos en 'marital_status': ['Widowed' 'Never married' 'Married-civilian spouse present' 'D

# Buscar filas duplicadas

In [164]:
duplicado = df[df.duplicated()]
duplicado

,ID,age,gender,education,class,education_institute,marital_status,race,is_hispanic,employment_commitment,...,country_of_birth_mother,migration_code_change_in_msa,migration_prev_sunbelt,migration_code_move_within_reg,migration_code_change_in_reg,residence_1_year_ago,old_residence_reg,old_residence_state,importance_of_record,income_above_limit


No hay filas duplicadas entonces no hay instances para remover

# Buscar nulos

In [167]:
df.isnull().sum()

ID                                     0
age                                    0
gender                                 0
education                              0
class                             105245
education_institute               196197
marital_status                         0
race                                   0
is_hispanic                            0
employment_commitment                  0
unemployment_reason               202979
employment_stat                        0
wage_per_hour                          0
is_labor_union                    189420
working_week_per_year                  0
industry_code                          0
industry_code_main                     0
occupation_code                        0
occupation_code_main              105694
total_employed                         0
household_stat                         0
household_summary                      0
under_18_family                   151654
veterans_admin_questionnaire      207415
vet_benefit     

In [168]:

# Crear una copia del dataset combinado
df_copia = df.copy()

# Lista de columnas a eliminar
columnas_a_eliminar = [
    'ID', 
    'is_labor_union', 
    'industry_code', 
    'occupation_code_main', 
    'household_summary',
    'under_18_family', 
    'veterans_admin_questionnaire', 
    'vet_benefit',
    'mig_year', 
    'country_of_birth_father', 
    'country_of_birth_mother', 
    'migration_prev_sunbelt', 
    'migration_code_move_within_reg',
    'residence_1_year_ago',
    'old_residence_reg',
    'old_residence_state', 
    'importance_of_record'
]

# Eliminar las columnas en la copia
df_copia = df_copia.drop(columns=columnas_a_eliminar, errors='ignore')

# Mostrar las primeras filas del dataset modificado
df_copia.head()


,age,gender,education,class,education_institute,marital_status,race,is_hispanic,employment_commitment,unemployment_reason,...,tax_status,gains,losses,stocks_status,citizenship,migration_year,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,income_above_limit
0,79,Female,High school graduate,None,None,Widowed,White,All other,Not in labor force,None,...,Head of household,0,0,292,Native,95,US,?,?,Below limit
1,65,Female,High school graduate,None,None,Widowed,White,All other,Children or Armed Forces,None,...,Single,0,0,0,Native,94,US,unchanged,unchanged,Below limit
2,21,Male,12th grade no diploma,Federal government,None,Never married,Black,All other,Children or Armed Forces,None,...,Single,0,0,0,Native,94,US,unchanged,unchanged,Below limit
3,2,Female,Children,None,None,Never married,Asian or Pacific Islander,All other,Children or Armed Forces,None,...,Nonfiler,0,0,0,Native,94,US,unchanged,unchanged,Below limit
4,70,Male,High school graduate,None,None,Married-civilian spouse present,White,All other,Not in labor force,None,...,Joint both 65+,0,0,0,Native,95,US,?,?,Below limit


# Filtrar columnas que tiene mas que 40% nulos

In [170]:
columnas_reducidas = df_copia.loc[:, (df_copia.isna().sum() / len(df_copia) * 100) < 40]
print(columnas_reducidas.shape)
print(columnas_reducidas.columns)

(209499, 24)
Index(['age', 'gender', 'education', 'marital_status', 'race', 'is_hispanic',
       'employment_commitment', 'employment_stat', 'wage_per_hour',
       'working_week_per_year', 'industry_code_main', 'occupation_code',
       'total_employed', 'household_stat', 'tax_status', 'gains', 'losses',
       'stocks_status', 'citizenship', 'migration_year',
       'country_of_birth_own', 'migration_code_change_in_msa',
       'migration_code_change_in_reg', 'income_above_limit'],
      dtype='object')


In [171]:
columnas_reducidas.shape

(209499, 24)

# Arreglar los nulos

De las columnas restantes, busque valores nulos. Los únicos valores nulos están en las columnas categóricas, por lo que se rellenan con la moda de la columna.

In [173]:
columnas_reducidas.isna().sum()

age                                0
gender                             0
education                          0
marital_status                     0
race                               0
is_hispanic                        0
employment_commitment              0
employment_stat                    0
wage_per_hour                      0
working_week_per_year              0
industry_code_main                 0
occupation_code                    0
total_employed                     0
household_stat                     0
tax_status                         0
gains                              0
losses                             0
stocks_status                      0
citizenship                        0
migration_year                     0
country_of_birth_own               0
migration_code_change_in_msa    1588
migration_code_change_in_reg    1588
income_above_limit                 0
dtype: int64

In [174]:
# ver values y el mas común 
print(columnas_reducidas['migration_code_change_in_msa'].unique())
print("Value mas comun:", columnas_reducidas['migration_code_change_in_msa'].mode()[0])

print(columnas_reducidas['migration_code_change_in_reg'].unique())
print("Value mas comun:", columnas_reducidas['migration_code_change_in_reg'].mode()[0])

['?' 'unchanged' 'MSA to MSA' None 'NonMSA to nonMSA' 'MSA to nonMSA'
 'Not identifiable' 'NonMSA to MSA' 'Abroad to MSA' 'Abroad to nonMSA']
Value mas comun: ?
['?' 'unchanged' 'Same county' 'Different state same division'
 'Different county same state' None 'Different region' 'Abroad'
 'Different division same region']
Value mas comun: ?


In [175]:
columns_to_process = [
    'migration_code_change_in_msa',
    'migration_code_change_in_reg'
]

for column in columns_to_process:
    if column in columnas_reducidas:
        columnas_reducidas.loc[:, column] = (
            columnas_reducidas[column]
            .replace({' ?': 'Unknown'})
            .fillna('Unknown')
        )


In [176]:
# verificar los valores
columnas_reducidas.isna().sum()

age                             0
gender                          0
education                       0
marital_status                  0
race                            0
is_hispanic                     0
employment_commitment           0
employment_stat                 0
wage_per_hour                   0
working_week_per_year           0
industry_code_main              0
occupation_code                 0
total_employed                  0
household_stat                  0
tax_status                      0
gains                           0
losses                          0
stocks_status                   0
citizenship                     0
migration_year                  0
country_of_birth_own            0
migration_code_change_in_msa    0
migration_code_change_in_reg    0
income_above_limit              0
dtype: int64

In [177]:
columnas_reducidas.shape

(209499, 24)

# Guardar output

In [204]:
columnas_reducidas.to_parquet("output_02.parquet")